In [15]:
from selenium import webdriver
import pandas as pd, json, requests
from selenium.webdriver.common.action_chains import ActionChains
from random import uniform
import time
from IPython.display import display, clear_output
from webdriver_manager.chrome import ChromeDriverManager
def remove_whereareyou_popup(driver):
    try:
        elm = driver.find_element_by_css_selector( "div [class*='WhereAreYouModal__Container']" )
        elm.find_element_by_tag_name('img').click()
    except selenium.common.exceptions.NoSuchElementException:
        print("no popup")
    return
def update_headers(driver,session):
    headers = {
    "User-Agent":
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
    }
    session.headers.update(headers)
    for cookie in driver.get_cookies():
        c = {cookie['name']: cookie['value']}
        session.cookies.update(c)
    return session
def remove_captcha(driver):
    def foo():
        try:
            obj = driver.find_element_by_css_selector( "div [role*='main']")
            ActionChains(driver).click_and_hold(obj).perform()
            time.sleep(3)
            ActionChains(driver).release(obj).perform()
            time.sleep(4)
        except selenium.common.exceptions.NoSuchElementException:
            print("[remove_captcha]:noSuchElementException")
            pass
    prev_size = driver.get_window_size()
    prev_w,prev_h = prev_size['width'],prev_size['height']
    driver.set_window_size(730, 1380)
    time.sleep(1)
    foo()
    driver.set_window_size(int(prev_w), int(prev_h))
    
    return 

def reset_session(driver=None,session=None):
    if driver:
        driver.close()
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://stockx.com')
    time.sleep(3)
    remove_captcha(driver)
    time.sleep(2)
    remove_whereareyou_popup(driver)
#     time.sleep(2)
#     log_in(driver)
    time.sleep(2)
    remove_captcha(driver)
    session = requests.session()
    session = update_headers(driver,session)
    return driver,session


def get_meta_info(tile):
    elms = tile.find_elements_by_tag_name('div')
    name = elms[3].text
    price_type = elms[6].text
    price = elms[7].text
    release_date = elms[8].text 
    main_url = tile.find_element_by_tag_name('a').get_attribute('href')
    
    return name,price_type,price,release_date,main_url
def load_general_info_detail(driver):
    for script in driver.find_elements_by_tag_name('script'):
        script = script.get_attribute('innerHTML').strip()
        if 'window.preLoaded = ' in script:
            data_json = json.loads(script[len('window.preLoaded = '):-1])
    return data_json
def get_pid(data_json):
    return data_json['product']['uuid']
def load_daliy_avg_bids(pid,session):
    chart_url_base = 'https://stockx.com/api/products/{pid}/chart?start_date={start_date}&end_date=2020-07-31&intervals={num_days}&format=highstock&currency=USD&country=KR'
    # find first bid
    bid_data = session.get(chart_url_base.format(pid=pid,num_days=1,start_date='all')).json()
    if bid_data!=None:
        first_bid_day = bid_data['series'][0]['data'][0][0]
        timedelta = (datetime.datetime.fromtimestamp(1596174754000/1e3)-datetime.datetime.fromtimestamp(first_bid_day/1e3)).days
        data = session.get(chart_url_base.format(pid=pid,num_days=timedelta,start_date='all'))
        data_list = data.json()['series'][0]['data']
    else:
        data_list = []
    return data_list
def get_detail_info(main_url,driver,session):
    driver.get(main_url)
    time.sleep(2)
    data_json = load_general_info_detail(driver)
    pid = get_pid(data_json)
    bid_list = load_daliy_avg_bids(pid,session)
    return data_json,pid,bid_list
def check_captcha(driver):
    try:
        obj = driver.find_element_by_css_selector( "div [role*='main']")
    except selenium.common.exceptions.NoSuchElementException:
        return 
    raise
    return 


"""
If you don't have Chrome, and chrome driver, please install it. 
1)
wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
2)
sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
3)
sudo apt-get update
4) 
sudo apt-get install google-chrome-stable
5) 
sudo rm -rf /etc/apt/sources.list.d/google.list
6)
google-chrome --version
7)
pip install webdriver_manager
8) 

"""
import pandas as pd # pip install pandas
import urllib # pip install urllib
META_PATH = '/home/sneakers/_data/val_sneakers_df_211206.pkl'
df = pd.read_pickle(META_PATH)
import os
IMAGE_FOLDER_PATH = 'images'
if not os.path.exists(IMAGE_FOLDER_PATH):
    os.mkdir(IMAGE_FOLDER_PATH)
    
driver,session = reset_session()
for idx, (main_url) in df[['main_url']].iterrows():
    data_json,pid,bid_list = get_detail_info(main_url=main_url,driver=driver,session=session)
    url = data_json['product']['media']['imageUrl']
    urllib.request.urlretrieve(url, IMAGE_FOLDER_PATH+"/{}.jpg".format(pid))

ModuleNotFoundError: No module named 'webdriver_manager.Chrome'

In [2]:
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -

OK


In [3]:
!sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'

In [4]:
!sudo apt-get update

Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,076 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,051 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,345 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,270 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,726 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,408 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [28.5 kB]
Ge

In [6]:
!sudo apt-get install -y google-chrome-stable

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core fontconfig fontconfig-config
  fonts-liberation gtk-update-icon-cache hicolor-icon-theme
  humanity-icon-theme libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data
  libatspi2.0-0 libauthen-sasl-perl libavahi-client3 libavahi-common-data
  libavahi-common3 libcairo-gobject2 libcairo2 libcolord2 libcups2
  libdata-dump-perl libdatrie1 libencode-locale-perl libepoxy0
  libfile-basedir-perl libfile-desktopentry-perl libfile-listing-perl
  libfile-mimeinfo-perl libfont-afm-perl libfontconfig1 libfontenc1
  libfreetype6 libfribidi0 libgbm1 libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin
  libgdk-pixbuf2.0-common libgraphite2-3 libgtk-3-0 libgtk-3-bin
  libgtk-3-common libharfbuzz0b libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libht

In [7]:
!sudo rm -rf /etc/apt/sources.list.d/google.list

In [14]:
%pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 17.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 28.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install urllib

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install IPython

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
